In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings


In [ ]:
#import data
df=pd.read_csv('/kaggle/input/t20i-cricket-score-prediction/t20I_cricket_dataset.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#Unnamed: 0 has no maning so remove it
df.drop('Unnamed: 0',axis=1,inplace=True)

**EDA**

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.select_dtypes('O').columns

# visualizations

In [ ]:
# visualization with Categorical columns

plt.figure(1,figsize=(12,9))
plt.subplots_adjust(wspace=0.32,hspace=.25)

plt.subplot(221)
sns.countplot(data=df,y='battingTeam',order = df['battingTeam'].value_counts().index)
plt.title('Number of Batting Team')

plt.subplot(222)
sns.countplot(data=df,y='bowlingTeam',order = df['bowlingTeam'].value_counts().index)
plt.title('Number of Bowling Team')

**highest number of batting and bowlling teams are from Pakistan**

In [ ]:

sns.barplot(y = df['city'].value_counts().head(10).index, x = df['city'].value_counts().head(10).values)
plt.title('Number of City')

**Most of the matches played in Dubai and Colombo.**

In [ ]:
# visualization with continious columns
df.select_dtypes('int').columns

In [ ]:
#finding outliers using kde subplots
plt.figure(3, figsize=(15, 8))
plt.subplots_adjust(wspace=0.1);

plt.subplot(221)
sns.kdeplot(data=df, x='score');

plt.subplot(222)
sns.boxplot(data=df,x='score',orient='h')

plt.subplot(223)
sns.kdeplot(data=df, x='CurrentRunRate');

plt.subplot(224)
sns.boxplot(data=df,x='CurrentRunRate',orient='h')


**Scores and Currentraunrate has some outliers.**

In [ ]:
#finding outliers using kde subplots
plt.figure(3, figsize=(15, 8))
plt.subplots_adjust(wspace=0.1);

plt.subplot(221)
sns.kdeplot(data=df, x='wicketsLeft');

plt.subplot(222)
sns.boxplot(data=df,x='wicketsLeft',orient='h')

plt.subplot(223)
sns.kdeplot(data=df, x='Run_In_Last5');

plt.subplot(224)
sns.boxplot(data=df,x='Run_In_Last5',orient='h')

**in above graph it showint Run_in_Last5 has outliers**

In [ ]:
#finding outliers using kde subplots
plt.figure(3, figsize=(15, 8))
plt.subplots_adjust(wspace=0.1);

plt.subplot(221)
sns.kdeplot(data=df, x='Wickets_In_Last5');

plt.subplot(222)
sns.boxplot(data=df,x='Wickets_In_Last5',orient='h')

plt.subplot(223)
sns.kdeplot(data=df, x='innings');

plt.subplot(224)
sns.boxplot(data=df,x='innings',orient='h')

**every team has approximetly 140 average score**

In [ ]:
#score scored by batting team (first batting)
f_score_btean=df.groupby('battingTeam').agg({'Final_Score':'mean'})
plt.figure(figsize=(10, 5))
sns.barplot(x = f_score_btean.index, y = f_score_btean['Final_Score'])
plt.xticks(rotation=30)
plt.show()
# f_score_btean


**India and England batting teams has the highest mean(160) of final score.**

In [ ]:
#score scored by bowling team (first bowlers)
f_score_btean=df.groupby('bowlingTeam').agg({'Final_Score':'mean'})
plt.figure(figsize=(10, 5))
sns.barplot(x = f_score_btean.index, y = f_score_btean['Final_Score'])
plt.xticks(rotation=30)
plt.show()


**Zimbabwe bolling teams has the highest mean(160) of final score.**

****

In [ ]:
#final score scored by each city played
df.groupby('city').agg({'Final_Score':'mean'}).sort_values('Final_Score',ascending=False).head(10).plot(kind='bar')

**mathes played in Denudin have the highest mean(220) of finalscore**

In [ ]:
#trying features combinations
df.groupby(['battingTeam','city']).agg({'Final_Score':'max'})

In [ ]:
#corelation between numerical column
metric=df.loc[:,['AverageScore', 'delivery_left', 'score', 'CurrentRunRate',
       'wicketsLeft', 'Run_In_Last5', 'Wickets_In_Last5', 'Final_Score',
       'innings']].corr()

In [ ]:
sns.heatmap(metric,annot=True)

# Outlier Handeling

**average score,current_runrate, score,runin_last5 has mostly outliers**

In [ ]:
#outlier visualisation
plt.figure(10, figsize=(18, 4))
plt.subplots_adjust(wspace=0.2)
sns.histplot(data=df, x='AverageScore', ax=plt.subplot(121),bins=10, kde=True, edgecolor='red');
sns.histplot(data=df, x='CurrentRunRate', ax=plt.subplot(122),bins=10, kde=True, edgecolor='red');

plt.figure(11, figsize=(18, 4))
plt.subplots_adjust(wspace=0.2)
sns.histplot(data=df, x='score', ax=plt.subplot(121),bins=10, kde=True, edgecolor='red');
sns.histplot(data=df, x='Run_In_Last5', ax=plt.subplot(122),bins=10, kde=True, edgecolor='red');
plt.show()


In [ ]:
# remove outliers
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply to all columns
for col in df.select_dtypes(include=['float', 'int']):
    df = remove_outliers_iqr(df, col)


In [ ]:
plt.figure(3, figsize=(15, 8))
plt.subplots_adjust(wspace=0.1);

plt.subplot(221)
sns.kdeplot(data=df, x='AverageScore');

plt.subplot(222)
sns.boxplot(data=df,x='AverageScore',orient='h')

**now,we can see there is no outliers in AverageScore.**

In [ ]:
obj_col=df.select_dtypes(include='object').columns
obj_col

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# for col in obj_col:
#   ohe=OneHotEncoder()
#   df[col]=ohe.fit_transform(df[col])

In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in obj_col:
  lb=LabelEncoder()
  df[col]=lb.fit_transform(df[col])

In [ ]:
df.head()

In [ ]:
# Extract indipendent and dependent Feature
x=df.drop('Final_Score',axis=1)
y=df.iloc[:,-2]

In [ ]:
#split the data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
print(f"X_train shape: {x_train.shape},X_test shape:{x_test.shape}")

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)

In [ ]:
y_pred=lr.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
mse=mean_squared_error(y_test,y_pred)
r1=r2_score(y_test,y_pred)

print('MSE : ',mse,'r2_score : ',r1)

In [ ]:
lr.score(x_train,y_train),lr.score(x_test,y_test)

# RandomForest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rgr=RandomForestRegressor()
rgr.fit(x_train,y_train)

In [ ]:
y_pred=rgr.predict(x_test)

In [ ]:
mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)
print('MSE : ',mse,'r2_score : ',r2)

In [ ]:
rgr.score(x_train,y_train),rgr.score(x_test,y_test)

# SVR

In [ ]:
from sklearn.svm import SVR
model_svr=SVR()
model_svr.fit(x_train,y_train)

In [ ]:
y_pred=model_svr.predict(x_test)

In [ ]:
mse=mean_squared_error(y_test,y_pred)
r3=r2_score(y_test,y_pred)
print('MSE : ',mse,'r2_score : ',r3)

# XGBoost

In [ ]:
from xgboost import XGBRegressor
model_xgb=XGBRegressor()
model_xgb.fit(x_train,y_train)

In [ ]:
y_pred=model_xgb.predict(x_test)

In [ ]:
mse=mean_squared_error(y_test,y_pred)
r4=r2_score(y_test,y_pred)
print('MSE : ',mse,'r2_score : ',r4)

In [ ]:
model_xgb.score(x_train,y_train),model_xgb.score(x_test,y_test)

In [ ]:
score=[r1,r2,r3,r4]
model=['LinearRegression','RandomForestRegression','SVR','XGBoost']



data_f=pd.DataFrame({'score':score,'model':model})
data_f

In [ ]:
sns.barplot(data=data_f,x=model,y=score)
plt.xticks(rotation=30)
plt.title('Accuracy Score')
plt.show()

**RandomForest and XGBoost performing good with accuracy score 0.86 and 0.91.**